#### Import the libraries

In [1]:
!pip install dataframe_image

In [2]:
!pip install pycaret

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import dataframe_image as dfi


In [4]:
from pycaret.classification import *
from sklearn.metrics import classification_report

In [5]:
df = pd.read_csv('/content/daset_ml_dl.csv', index_col=0)
#df_type = df['Type']
df.shape

(11526, 47)

#### Remove duplicates

In [6]:
duplicate_rows = df[df.duplicated()]
len(duplicate_rows)

0

In [7]:
df.columns.values

array(['project', 'DPy', 'PySmell', 'file_name', 'method_name',
       'analyse_type', 'code_metric', 'raw_loc', 'raw_lloc', 'raw_sloc',
       'raw_comments', 'raw_multi', 'raw_blank', 'raw_single_comments',
       'cc_type', 'cc_rank', 'cc_complexity', 'cc_name', 'cc_lineno',
       'cc_closures', 'mi_mi', ' mi_rank', 'hal_t_h1', 'hal_t_h2',
       'hal_t_N1', 'hal_t_N2', 'hal_t_vocabulary', 'hal_t_length',
       'hal_t_calculated_length', 'hal_t_volume', 'hal_t_difficulty',
       'hal_t_effort', 'hal_t_time', 'hal_t_bugs', 'hal_func_h1',
       'hal_func_h2', 'hal_func_N1', 'hal_func_N2', 'hal_func_vocabulary',
       'hal_func_length', 'hal_func_calculated_length', 'hal_func_volume',
       'hal_func_difficulty', 'hal_func_effort', 'hal_func_time',
       'hal_func_bugs', 'Unnamed: 0'], dtype=object)

In [8]:
num_colunas = len(df.columns)
print(num_colunas)

47


## Drop Code Smell

##### Drop the code smells that are not necessary for the model. For instance, this code is set up to the Long Method smell. To train other models, remove the other code smells.

#### Check the correlation and remove the high-correlated features from the dataset

In [9]:
target = df['analyse_type']
corr = df.corr(numeric_only = True)
threshold = 0.8
#columns = np.full((corr.shape[0],), True,dtype='object')
columns = np.isin(df.columns,corr.index)
for i in range(corr.shape[0]):
    for j in range(i+1, corr.shape[0]):
        if corr.iloc[i,j] >= threshold:
            if columns[j]:
                columns[j] = False
selected_columns = df.columns[columns]
high_corr = set(df.columns) - set(selected_columns)
df = df[selected_columns]
df['analyse_type'] = target
df.columns
# Separar target antes da filtragem



Index(['DPy', 'PySmell', 'raw_lloc', 'raw_comments', 'raw_multi', 'raw_blank',
       'raw_single_comments', 'hal_func_N2', 'hal_func_vocabulary',
       'hal_func_length', 'hal_func_calculated_length', 'hal_func_volume',
       'hal_func_difficulty', 'hal_func_effort', 'hal_func_time',
       'hal_func_bugs', 'Unnamed: 0', 'analyse_type'],
      dtype='object')

In [10]:
df

,DPy,PySmell,raw_lloc,raw_comments,raw_multi,raw_blank,raw_single_comments,hal_func_N2,hal_func_vocabulary,hal_func_length,hal_func_calculated_length,hal_func_volume,hal_func_difficulty,hal_func_effort,hal_func_time,hal_func_bugs,Unnamed: 0,analyse_type
0,0,0,9,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,long-parameter-list
1,0,0,2,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,long-parameter-list
2,0,0,3,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,long-parameter-list
3,0,0,3,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,long-parameter-list
4,0,0,29,0,0,0,0,13,17,21,54.629190,85.836720,2.708333,232.474449,12.915247,0.028612,NaN,long-parameter-list
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
825,1,1,2,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,825.0,long-method
826,1,1,2,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,826.0,long-method
827,1,1,2,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,827.0,long-method
828,1,1,2,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,828.0,long-method


In [11]:
# Substituir caracteres especiais por underline e remover espaços
df.columns = df.columns.str.replace('[^A-Za-z0-9_]+', '_', regex=True).str.strip()

In [12]:
df

,DPy,PySmell,raw_lloc,raw_comments,raw_multi,raw_blank,raw_single_comments,hal_func_N2,hal_func_vocabulary,hal_func_length,hal_func_calculated_length,hal_func_volume,hal_func_difficulty,hal_func_effort,hal_func_time,hal_func_bugs,Unnamed_0,analyse_type
0,0,0,9,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,long-parameter-list
1,0,0,2,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,long-parameter-list
2,0,0,3,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,long-parameter-list
3,0,0,3,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,long-parameter-list
4,0,0,29,0,0,0,0,13,17,21,54.629190,85.836720,2.708333,232.474449,12.915247,0.028612,NaN,long-parameter-list
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
825,1,1,2,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,825.0,long-method
826,1,1,2,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,826.0,long-method
827,1,1,2,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,827.0,long-method
828,1,1,2,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,828.0,long-method


##### Put the target on the end of the dataframe. Here, as an example, our target is the presence of Long Method.

#### Setup the model creation with pycaret

In [13]:
s = setup(data=df, target='analyse_type', feature_selection=True,
          remove_multicollinearity=True,
          multicollinearity_threshold=0.85,
          feature_selection_method='classic',
          fix_imbalance=True, fold=10,index=False)

[LightGBM] [Info] Number of positive: 8068, number of negative: 8068
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002368 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1438
[LightGBM] [Info] Number of data points in the train set: 16136, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


,Description,Value
0,Session id,2289
1,Target,analyse_type
2,Target type,Binary
3,Target mapping,"long-method: 0, long-parameter-list: 1"
4,Original data shape,"(13186, 18)"
5,Transformed data shape,"(20092, 4)"
6,Transformed train set shape,"(16136, 4)"
7,Transformed test set shape,"(3956, 4)"
8,Numeric features,17
9,Rows with missing values,87.4%


In [14]:
df = get_config('X')
df

,DPy,PySmell,raw_lloc,raw_comments,raw_multi,raw_blank,raw_single_comments,hal_func_N2,hal_func_vocabulary,hal_func_length,hal_func_calculated_length,hal_func_volume,hal_func_difficulty,hal_func_effort,hal_func_time,hal_func_bugs,Unnamed_0
0,0,0,4,0,0,0,0,2,3,3,2.000000,4.754888,0.500000,2.377444,0.132080,0.001585,NaN
1,0,0,2,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
2,0,0,16,0,0,0,0,3,5,5,6.754888,11.609640,1.000000,11.609640,0.644980,0.003870,NaN
3,0,0,3,0,4,1,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
4,0,0,3,0,0,0,1,6,7,9,13.609640,25.266193,1.200000,30.319433,1.684413,0.008422,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13181,0,0,8,0,0,0,0,69,38,104,184.279800,545.784485,2.957143,1613.962646,89.664589,0.181928,NaN
13182,0,0,7,0,0,0,0,2,3,3,2.000000,4.754888,0.500000,2.377444,0.132080,0.001585,NaN
13183,0,0,6,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
13184,0,0,2,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN


##### Train the models. In this code the model was trained for the Long Method smell. From all models, we selected the top-5 in terms of F1.

In [15]:
top5_models = compare_models(n_select=5, sort='f1')

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.8480,0.5810,0.8480,0.8165,0.8286,0.1479,0.1568,1.0720
lightgbm,Light Gradient Boosting Machine,0.8393,0.5623,0.8393,0.8098,0.8216,0.1221,0.1287,1.7550
nb,Naive Bayes,0.8320,0.5462,0.8320,0.8065,0.8178,0.1150,0.1178,0.2760
ada,Ada Boost Classifier,0.8258,0.5672,0.8258,0.8144,0.8177,0.1430,0.1495,0.6820
qda,Quadratic Discriminant Analysis,0.8320,0.5450,0.8320,0.8061,0.8176,0.1133,0.1160,0.2730
xgboost,Extreme Gradient Boosting,0.8135,0.5518,0.8135,0.7945,0.8032,0.0645,0.0655,0.3830
knn,K Neighbors Classifier,0.8004,0.5679,0.8004,0.8005,0.8001,0.0927,0.0930,0.4130
rf,Random Forest Classifier,0.8053,0.5030,0.8053,0.7823,0.7929,0.0095,0.0101,1.3460
et,Extra Trees Classifier,0.7958,0.4567,0.7958,0.7783,0.7866,-0.0070,-0.0070,1.1430
ridge,Ridge Classifier,0.7761,0.5471,0.7761,0.7942,0.7847,0.0640,0.0643,0.2760


Processing:   0%|          | 0/69 [00:00<?, ?it/s]

In [16]:
df2=pull()
df3 = pd.DataFrame(df2)
df3.to_latex('Multilabel_geral.latex',index=False)
df3.to_csv('Multilabel_geral.csv',index=False)
#dfi.export(df2, 'Bin_Blob_FS1.png')

##### Result by Smell

In [17]:
from sklearn.metrics import classification_report
from pycaret.classification import predict_model

# Dicionário para armazenar os relatórios por modelo e por classe
reports = {}
# Lista para armazenar os dados
data = []

# Iterar sobre os modelos
for model in top5_models:
    # Gerar predições com o modelo individual
    predictions = predict_model(model)

    # Obter nome do modelo
    model_name = model.__class__.__name__

    # Gerar relatório por classe
    report = classification_report(
        predictions['analyse_type'],
        predictions['prediction_label'],
        output_dict=True
    )

    # Armazenar o relatório
    reports[model_name] = report

    # Exibir e armazenar métricas por classe
    print(f"\n📊 Classification Report for {model_name}")
    for smell_class in ['long-method', 'long-parameter-list']:
        if smell_class in report:
            print(f"\nSmell: {smell_class}")
            print(f"  Accuracy: {report['accuracy']:.2f}")
            print(f"  Precision: {report[smell_class]['precision']:.2f}")
            print(f"  Recall: {report[smell_class]['recall']:.2f}")
            print(f"  F1-score: {report[smell_class]['f1-score']:.2f}")

            # Armazenar para eventual DataFrame
            data.append({
                "Model": model_name,
                "Smell": smell_class,
                "Accuracy": report["accuracy"],
                "Precision": report[smell_class]["precision"],
                "Recall": report[smell_class]["recall"],
                "F1-score": report[smell_class]["f1-score"]
            })
    print("=" * 80)


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Gradient Boosting Classifier,0.8630,0.5869,0.8630,0.8242,0.8355,0.1553,0.1767



📊 Classification Report for GradientBoostingClassifier

Smell: long-method
  Accuracy: 0.86
  Precision: 0.39
  Recall: 0.15
  F1-score: 0.21

Smell: long-parameter-list
  Accuracy: 0.86
  Precision: 0.89
  Recall: 0.97
  F1-score: 0.92


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Light Gradient Boosting Machine,0.8516,0.5888,0.8516,0.8160,0.8295,0.1422,0.1527



📊 Classification Report for LGBMClassifier

Smell: long-method
  Accuracy: 0.85
  Precision: 0.32
  Recall: 0.16
  F1-score: 0.21

Smell: long-parameter-list
  Accuracy: 0.85
  Precision: 0.89
  Recall: 0.95
  F1-score: 0.92


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Naive Bayes,0.8552,0.5730,0.8552,0.8159,0.8298,0.1343,0.1481



📊 Classification Report for GaussianNB

Smell: long-method
  Accuracy: 0.86
  Precision: 0.33
  Recall: 0.14
  F1-score: 0.20

Smell: long-parameter-list
  Accuracy: 0.86
  Precision: 0.89
  Recall: 0.96
  F1-score: 0.92


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Ada Boost Classifier,0.8488,0.5734,0.8488,0.8260,0.8355,0.1988,0.2039



📊 Classification Report for AdaBoostClassifier

Smell: long-method
  Accuracy: 0.85
  Precision: 0.35
  Recall: 0.23
  F1-score: 0.28

Smell: long-parameter-list
  Accuracy: 0.85
  Precision: 0.89
  Recall: 0.94
  F1-score: 0.92


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Quadratic Discriminant Analysis,0.8547,0.5663,0.8547,0.8150,0.8292,0.1312,0.1447



📊 Classification Report for QuadraticDiscriminantAnalysis

Smell: long-method
  Accuracy: 0.85
  Precision: 0.32
  Recall: 0.14
  F1-score: 0.20

Smell: long-parameter-list
  Accuracy: 0.85
  Precision: 0.89
  Recall: 0.96
  F1-score: 0.92


In [18]:
# Criar um DataFrame
df = pd.DataFrame(data)

# Salvar o DataFrame em um arquivo CSV
df.to_csv("smell_classification_Multilabel.csv", index=False)

In [19]:
df2=pull()
df3 = pd.DataFrame(df2)
df3.to_latex('multilabel_.latex',index=False)
df3.to_csv('multilabel.csv',index=False)

#### Ensemble the top-5 models

In [20]:
blended_models = blend_models(
  top5_models,
  choose_better=True,
  optimize="f1")

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8310,0.5912,0.8310,0.8062,0.8173,0.1131,0.1155
1,0.8375,0.5515,0.8375,0.8010,0.8165,0.0849,0.0895
2,0.8678,0.6401,0.8678,0.8387,0.8471,0.2275,0.2459
3,0.8527,0.5287,0.8527,0.8159,0.8296,0.1388,0.1501
4,0.8559,0.6145,0.8559,0.8170,0.8307,0.1374,0.1515
5,0.8332,0.5043,0.8332,0.7930,0.8105,0.0518,0.0549
6,0.8494,0.6437,0.8494,0.8102,0.8253,0.1156,0.1255
7,0.8429,0.5773,0.8429,0.8170,0.8280,0.1577,0.1624
8,0.8451,0.5977,0.8451,0.8173,0.8287,0.1625,0.1685


Processing:   0%|          | 0/6 [00:00<?, ?it/s]

Original model was better than the blended model, hence it will be returned. NOTE: The display metrics are for the blended model (not the original one).


In [21]:
pred = predict_model(blended_models)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Gradient Boosting Classifier,0.8630,0.5869,0.8630,0.8242,0.8355,0.1553,0.1767


In [22]:
result = pull()

#### Plot the ensemble model

In [23]:
result.to_csv('multilabel_ensemble.csv')

#### Finalize and save the ensemble model

In [24]:
final_best = finalize_model(blended_models)
save_model(final_best, 'multilabel')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('label_encoding',
                  TransformerWrapperWithInverse(exclude=None, include=None,
                                                transformer=LabelEncoder())),
                 ('numerical_imputer',
                  TransformerWrapper(exclude=None,
                                     include=['DPy', 'PySmell', 'raw_lloc',
                                              'raw_comments', 'raw_multi',
                                              'raw_blank', 'raw_single_comments',
                                              'hal_func_N2',
                                              'hal_func_vocabulary',
                                              'hal_func_length...
                                             criterion='friedman_mse', init=None,
                                             learning_rate=0.1, loss='log_loss',
                                             max_depth=3, max_features=None,
           

##### Drop the code smells that are not necessary for the model. For instance, this code is set up to the Long Method smell. To train other models, remove the other code smells.